In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import linear_model,model_selection,metrics
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train,test = pd.read_csv('../input/train.csv'),pd.read_csv('../input/test.csv')
text = pd.concat([train,test],sort=False)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    lowercase=True,
    token_pattern=r'\w{1,}',
    stop_words='english',
    #max_features=9000,
    max_df = 0.95,
    min_df = 2
    
)

In [ ]:
vect = tfidf_vectorizer.fit_transform(text['question_text'])
train_v,test_v = vect[:1306122],vect[1306122:]
train_X,val_X = model_selection.train_test_split(train_v,test_size=0.2,random_state=42 )
train_Y,val_Y = model_selection.train_test_split(train.target,test_size=0.2,random_state=42)#more or less unsafe

In [ ]:
def multiple_log_reg_model(train_X,train_Y,test_X,test_Y):
    clf = linear_model.LogisticRegression(random_state=42,solver='lbfgs',multi_class='multinomial').fit(train_X,train_Y)
    pred = clf.predict(test_X)
    
    f1 = metrics.f1_score(test_Y,pred,average='weighted')
    test_score,train_score = clf.score(train_X,train_Y),clf.score(test_X,test_Y)
    
    print("f1: {0}, test score: {1}, train score: {2}".format(f1,test_score,train_score))
    
    return clf

In [ ]:
clf = multiple_log_reg_model(train_X,train_Y,val_X,val_Y)
pred_test_y = clf.predict(test_v)
pred_test_labels = (pred_test_y>0.5).astype(int)

In [ ]:
out_df = pd.DataFrame({"qid":test.qid.values})
out_df['prediction'] = pred_test_labels
out_df.to_csv("submission.csv",index=False)